In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [2]:
df_01 = pd.read_parquet('fhv_tripdata_2021-01.parquet')
df_02 = pd.read_parquet('fhv_tripdata_2021-02.parquet')

### 1. Number of records in Jan 2021 FHV data

In [3]:
df_01.shape

(1154112, 7)

calculate duration

In [4]:
def calc_duration(df):
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda x: x.total_seconds()/60)
    
    return df

In [5]:
df_01 = calc_duration(df_01)
df_02 = calc_duration(df_02)

### 2. Average duration in Jan 2021 FHV

In [6]:
df_01.duration.mean()

19.1672240937939

### Data preparation

In [7]:
df_train = df_01[((df_01.duration >= 1) & (df_01.duration <= 60))]
df_val = df_02[((df_02.duration >= 1) & (df_02.duration <= 60))]

In [8]:
df_train.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'duration'],
      dtype='object')

In [9]:
columns = ['PUlocationID', 'DOlocationID']
df_train[columns ].isnull().sum() / df_train[columns].shape[0]

PUlocationID    0.835273
DOlocationID    0.133270
dtype: float64

In [10]:
df_train[columns] = df_train[columns].fillna(-1, axis=0)
df_val[columns]  = df_val[columns].fillna(-1, axis=0)

C:\Users\evgemikhaylov\AppData\Local\Temp\ipykernel_21188\962798131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns] = df_train[columns].fillna(-1, axis=0)
C:\Users\evgemikhaylov\AppData\Local\Temp\ipykernel_21188\962798131.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[columns]  = df_val[columns].fillna(-1, axis=0)


In [11]:
dict_df = df_train[columns].astype(str).to_dict('records')
dv = DictVectorizer()
X_train = dv.fit_transform(dict_df)

y_train = df_train['duration'].values

In [13]:
dict_df_val = df_val[columns].astype(str).to_dict('records')
dv_v = DictVectorizer()
X_val = dv_v.fit_transform(dict_df_val)

y_val= df_val['duration'].values

In [14]:
X_train.shape

(1109826, 525)

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

x_pred = lr.predict(X_train)
mean_squared_error(y_train, x_pred, squared=False)

10.5285191072048

In [16]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

ValueError: X has 526 features, but LinearRegression is expecting 525 features as input.